#Main file
This main file is based on the scripts that I had for minimizing the machines from Netlogo. The original was made to capture output from Behaviour Space, processs the machines and then print it to use in Stata. This is too cumbersome, so decided to implement and do everything in Python so I can centralise all the analysis and work on the next algorithms such as Joint Machines, frequencies and unused behavioural states in order to analyse properly how the transitions are happening.

1) Run the first cell (to import libraries and load the minimization algorithms)
2) In the next cell, decide the path where the netlogo output (in "Table" form) is. Comment everything in that cell except one file
3) Run next cell (MAIN CODE)
4) Decide path to save the file, and run final cell

In [17]:
import numpy as np
import pandas as pd
import copy

# Big part of this code (canonical_automaton, remamp function and minimized_auto)is based on Warren's
# version of Jhon Miller's code, originally in Java.

# Create an empty auto as a numpy array
def new_empty_auto():
    dtype = [('actions', 'S1'), ('transitions', np.int32, n_obs)] # structure for "normal_auto" variable
    new_auto = np.zeros(n_states, dtype) # initialize normal_auto (make all transitions zero)
    new_auto['actions'] = 'x' # Make all actions x
    return new_auto
    
    
    
# return a cannonically ordered auto stripped of inaccessible states
def convert_to_canonical(normal_auto):
    # statemap will keep track of which states are remapped, and the order in which they should be
    # nextstate tracks the next available state number and eventually provides the number of accessible states
    
    statemap = np.ones(n_states)*(-1) #initialize map with null values
    statemap[init_state] = 0 # initial state renumbered to state 0
    nextstate = 1 # number of states remapped 
    # start it off with the start state, off to recursive remapper
    nextstate = remap(init_state, normal_auto['transitions'], statemap, nextstate) # nextstate is the number of states remapped
    
    # Here, after the recursion of the 'remap' function, the two key variables obtained are nextstates and statemap
    # nextstates was used to build statemap, but it contains the number of available states on the machine
    # statemap shows which are those states, and in which order they are accessed.
    
    #print 'nextstate = ', nextstate
    #print 'statemap (for canonical)= ', statemap
    
    auto = new_empty_auto() # Create new empty auto
    for s in xrange(n_states):
        if statemap[s] >= 0:   #if state is remapped (and accesible) 
            auto[statemap[s]]['actions'] = normal_auto[s]['actions']
            for t in xrange(n_obs):
                auto[statemap[s]]['transitions'][t] = statemap[normal_auto[s]['transitions'][t]] 
    auto = auto[:nextstate] # Cuts the auto to contain only the accesible states
    
    global updated_n_states # Used to "get out" of this function the local variable nexstate
    updated_n_states = nextstate # To update n_states. Use this because nexstate is local variable
    return auto


# Function 'remap' is used to convert into canonical form ('convert_to_canonical')
# ***recursively*** branches down automata and remaps everything via the ordered inputs
def remap(state0, transitions, statemap, nextstate):
    for t in xrange(n_obs): # for all possible observation/input
        if statemap[transitions[state0][t]] < 0: # if that state is not yet remapped (i.e. is not -1)
            statemap[transitions[state0][t]] = nextstate # assign it next available state num, then inc nstate (next line is the increase)
            nextstate += 1
            nextstate = remap(transitions[state0][t], transitions, statemap, nextstate) # recursively remap on this state
    return nextstate  

def minimized_automaton():
    # define the equivalence matrix
    equiv = np.zeros((n_states, n_states), dtype = bool)
    for s1 in xrange(n_states): #All possible states pair combinations
        for s2 in xrange(s1, n_states): 
            if canon_auto[s1]['actions'] == canon_auto[s2]['actions']: # Mark potentially equivalent states (i.e. with same action)
                equiv[s1][s2] = True
                equiv[s2][s1] = True # probably not needed, but cost is low
    #print equiv
    
    # now refine the equivalence matrix by iterating transitions on transitions until it stablizes           
    while True:
        changed = False # Track if changes ocurred during the iteration. If no changes, we are done!
        newequiv = np.zeros((n_states, n_states), dtype = bool) # Will contain the new equivalence matrix
        for s1 in xrange(n_states): # All possible states pair combinations
            for s2 in xrange(s1, n_states):
                if equiv[s1][s2]: # for all potential equivalent states (i.e. states with same action)                  
                    sametransitions = True # Assume they have the same transitions
                    for t in xrange(n_obs): # for all transitions
                        # Compares if the transitions in both states are the same (i.e. lead to the same action)
                        if (not equiv[canon_auto[s1]['transitions'][t]][canon_auto[s2]['transitions'][t]]):
                            sametransitions = False           
                    newequiv[s1][s2] = sametransitions
                    newequiv[s2][s1] = sametransitions
                    if (not sametransitions): 
                        changed = True # At least one change was made, so iterate again
        equiv = newequiv # Update the equivalence matrix to the modified one                         
        if not changed: # if changed is False (no changes ocurred), then exit the loop
            break

    # equiv now holds the truly equivalent states, so remap these into a new, minimized automaton
    # Make a new statemap to copy the new auto
    statemap = np.array(xrange(n_states)) # initial statemap just maps to current state
    for s1 in xrange(n_states):
        for s2 in xrange(s1+1, n_states):
            if equiv[s1][s2]:
                statemap[s2] = statemap[s1]
    #statemap[init_state] = 0 # I think not needed
    #print 'statemap (for minimization) = ', statemap

    # This looks like the same procedure used for copying the canonical auto (uses the new statemap)
    newauto = copy.deepcopy(canon_auto)
    #print 'newauto', newauto
    for s in xrange(n_states):
        newauto[statemap[s]]['actions'] = canon_auto[s]['actions']
        for t in xrange(n_obs):
            newauto[statemap[s]]['transitions'][t] = statemap[canon_auto[s]['transitions'][t]]
    newauto = convert_to_canonical(newauto)
    return newauto 


In [18]:
#Choose file to work with. Comment all except the one to use

# Import .csv file given by Netlogo. skiprows used to skip imkporting first rows, which are Netlogo info (not variable)

# Path in my WINDOWS desktop
data = pd.read_csv('C:/Users/lexlale/Dropbox/Thesis Phd/Coordination autos Chapter three/Netlogo original files/\
Autos-Coordination 8S 100,000 periods-table (3 of 3).csv', skiprows = 6)

In [19]:

############################################
############################################
############################################
# MAIN code follows
############################################
############################################

#Define new labels for the variables. Careful that the order of the reporters in the Netlogo file don't change,
# otherwise the variables would be mixed up
new_variable_labels = ['run_number',
'n_states',
'n-signal-cards',
'game_played',
'n_rounds',
'n_populations',
'n_outputs',
'N',
'n_signals',
'N_parents',
'generation',
'av_score_row',
'av_score_column',
'av_score',
'parent_col_0',
'parent_col_1',
'parent_col_2',
'parent_col_3',
'parent_col_4',
'parent_col_5',
'parent_col_6',
'parent_col_7',
'parent_col_8',
'parent_col_9',
'parent_col_10',
'parent_col_11',
'parent_col_12',
'parent_col_13',
'parent_col_14',
'parent_col_15',
'parent_col_16',
'parent_col_17',
'parent_col_18',
'parent_col_19',
'offspring_col_0',
'offspring_col_1',
'offspring_col_2',
'offspring_col_3',
'offspring_col_4',
'offspring_col_5',
'offspring_col_6',
'offspring_col_7',
'offspring_col_8',
'offspring_col_9',
'offspring_col_10',
'offspring_col_11',
'offspring_col_12',
'offspring_col_13',
'offspring_col_14',
'offspring_col_15',
'offspring_col_16',
'offspring_col_17',
'offspring_col_18',
'offspring_col_19',
'parent_row_0',
'parent_row_1',
'parent_row_2',
'parent_row_3',
'parent_row_4',
'parent_row_5',
'parent_row_6',
'parent_row_7',
'parent_row_8',
'parent_row_9',
'parent_row_10',
'parent_row_11',
'parent_row_12',
'parent_row_13',
'parent_row_14',
'parent_row_15',
'parent_row_16',
'parent_row_17',
'parent_row_18',
'parent_row_19',
'offspring_row_0',
'offspring_row_1',
'offspring_row_2',
'offspring_row_3',
'offspring_row_4',
'offspring_row_5',
'offspring_row_6',
'offspring_row_7',
'offspring_row_8',
'offspring_row_9',
'offspring_row_10',
'offspring_row_11',
'offspring_row_12',
'offspring_row_13',
'offspring_row_14',
'offspring_row_15',
'offspring_row_16',
'offspring_row_17',
'offspring_row_18',
'offspring_row_19',
'mutants_row',
'mutants_column',
'times_of_miscoordination',
'times_of_row_preference',
'times_of_column_preference']

data.columns = new_variable_labels # Assign new labels as variable names (columns of the dataframe)

# Make sure to include here all the columns with autos to minimize.

cols = ['parent_col_0',
'parent_col_1',
'parent_col_2',
'parent_col_3',
'parent_col_4',
'parent_col_5',
'parent_col_6',
'parent_col_7',
'parent_col_8',
'parent_col_9',
'parent_col_10',
'parent_col_11',
'parent_col_12',
'parent_col_13',
'parent_col_14',
'parent_col_15',
'parent_col_16',
'parent_col_17',
'parent_col_18',
'parent_col_19',
'offspring_col_0',
'offspring_col_1',
'offspring_col_2',
'offspring_col_3',
'offspring_col_4',
'offspring_col_5',
'offspring_col_6',
'offspring_col_7',
'offspring_col_8',
'offspring_col_9',
'offspring_col_10',
'offspring_col_11',
'offspring_col_12',
'offspring_col_13',
'offspring_col_14',
'offspring_col_15',
'offspring_col_16',
'offspring_col_17',
'offspring_col_18',
'offspring_col_19',
'parent_row_0',
'parent_row_1',
'parent_row_2',
'parent_row_3',
'parent_row_4',
'parent_row_5',
'parent_row_6',
'parent_row_7',
'parent_row_8',
'parent_row_9',
'parent_row_10',
'parent_row_11',
'parent_row_12',
'parent_row_13',
'parent_row_14',
'parent_row_15',
'parent_row_16',
'parent_row_17',
'parent_row_18',
'parent_row_19',
'offspring_row_0',
'offspring_row_1',
'offspring_row_2',
'offspring_row_3',
'offspring_row_4',
'offspring_row_5',
'offspring_row_6',
'offspring_row_7',
'offspring_row_8',
'offspring_row_9',
'offspring_row_10',
'offspring_row_11',
'offspring_row_12',
'offspring_row_13',
'offspring_row_14',
'offspring_row_15',
'offspring_row_16',
'offspring_row_17',
'offspring_row_18',
'offspring_row_19']

# This basically drops the first two observations per run of the simulation (tick==0 and 1)
data = data[data['generation'] != 0]  # Drop observations (rows) for which some autos are not reported
data = data[data['generation'] != 1]

# Uncomment next line to make it run on selected columns (for testing)
#cols = ['parent_col_1', 'parent_row_1'] # To test and use only few columns
#temporal_index = [10]

#for each column containing autos (cols), and each round (data.index)
for var in cols:
    label = 'min_' + var
    data[label] = np.nan # New column to insert minimized auto for each parent
    for ix in data.index: # Change "temporal_index" for "data.index" for the full run (all rows)
        big_auto = data[var][ix] #Choose one auto (Netlogo passes it, in my code, in Unicode format)
        
        # Next block is to clean the auto, because from Netlogo (Excel file) it comes formatted
        big_auto = big_auto.encode("ascii") # Convert to string
        #print type(big_auto) # Should be string
        big_auto = big_auto.replace('[','') # Delete useless characters
        big_auto = big_auto.replace(']','')
        big_auto = big_auto.replace('"','')
        big_auto = big_auto.split(' ') # Converts the string into a list
        
        for i in xrange(len(big_auto)): # This "for" converts all the numbers (transitions) into integers
            if big_auto[i] != 'A' and big_auto[i] != 'B':
                big_auto[i] = int(big_auto[i])
        #print 'original auto = ', big_auto, type(big_auto)
        
        # Next block saves some variables of the auto (number of states, number of observations, etc.)
        n_states = data['n_states'][ix] # Number of internal states of the auto
        #print 'number of states = ', n_states
        # The possible observations depend on the number of signals and the possible output of the machine
        # If no signal, can only observe rival playing A or B. Otherwise (i.e. one signal), is A or B for each signal (Heads or Tails)
        n_obs = 2 if data['n_signals'][ix] == 0 else 4
        #print 'number of observations = ', n_obs
        init_state = big_auto[0] # First item in the string represents the initial state
        #print 'initial state = ', init_state
        
        normal_auto = new_empty_auto() # Create the auto as a numpy array.
        
        # Fill the new 'normal_auto' with the information from big_auto. So normal_auto=big_auto but as an array
        my_index = xrange(1, len(big_auto), n_obs + 1) # Each number in the index is where a state starts
        for i, j in enumerate(my_index):
            normal_auto['actions'][i] = big_auto[j]
            normal_auto['transitions'][i] = big_auto[j + 1:j + n_obs + 1]
        #print 'auto as an array = ', normal_auto, type(normal_auto)
        
        # Convert to canonical form
        canon_auto = convert_to_canonical(normal_auto) # returns normal_auto in canonical form, and assigns it to canon_auto
        #print 'auto in canon form = ', canon_auto
        
        n_states = updated_n_states # n_states now is only the accesible states
        init_state = 0 # After in canonical form , the initial state is always 0 (by definition)
        #print 'n_states = ', n_states
        
        min_auto = minimized_automaton() # Now minimize the auto
        #print 'min_auto = ', min_auto
        
        print_auto = np.array_str(min_auto) # The min_auto is passed as a string (not an array) for easy printing
        
        print_auto = print_auto.replace('[','') # Make the string less cluttered
        print_auto = print_auto.replace(']','')
        print_auto = print_auto.replace("'",'')
        print_auto = print_auto.replace(',',' ')
        print_auto = print_auto.replace('\n', '')
        #print 'print_auto = ', print_auto
        
        data.loc[ix, label] = print_auto # Print the minimized auto as a string into the dataframe
        
sorted_df = data.sort(['run_number', 'generation']) #Sort the data





In [20]:
#Path and name of the file with the minimized autos to be saved

#Path for WINDOWS desktop office
sorted_df.to_excel('C:/Users/lexlale/Dropbox/Thesis Phd/Coordination autos Chapter three/\
8S 100,000 periods-table (3 of 3).xls')